# optim
参考：[module-torch.optim](https://pytorch.org/docs/stable/optim.html)

简单明了解释：包含不同的优化算法更新参数的软件包。接口通用，还可以集成更复杂算法（自定义）。

# Constructing it
要构造一个Optimizer，必须为其提供一个包含参数的可迭代项（所有参数都应为Variables）以进行优化。然后，可以指定优化器特定的选项，例如学习率，权重衰减等。

tips：使用`.cuda()`时，需要保持位置一致。

例如
```python
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam([var1, var2], lr=0.0001)
```

# per-parameter options
`optimizer`支持指定参数，通过传入可迭代的`dict`，且包含一个`params`key，对应的需要优化的参数列表。

ps:同样可以用关键字指定

```python
optim.SGD([
    {'params': model.base.parameters()},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
    ], lr=1e-2, momentum=0.9)
```

`model.base`的参数以默认的`1e-2`，而`model.classifier`参数以学习率`1e-3`优化。


# Taking an optimization step
所有的优化器都实现了一个方法:`.step()`，当`loss.backward()`被调用后立即调用以更新参数。

包括了2个调用形式

## 简单版本：optimizer.step()
```python
for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
```

## 闭包版本：optimizer.step(closure)
一些优化算法（例如共轭梯度和LBFGS）需要多次重新评估函数，因此必须传递一个闭包，以使它们可以重新计算模型。闭合应清除梯度，计算loss，然后将其返回。

```python
for input, target in dataset:
    def closure():
        optimizer.zero_grad()
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        return loss
    optimizer.step(closure)
```

# 优化算法
中文参考：[torch.optim](https://pytorch.apachecn.org/docs/1.4/87.html)\
英文参考：[optim](https://pytorch.org/docs/stable/optim.html)

参考知乎：[PyTorch 学习笔记（七）：PyTorch的十个优化器](https://zhuanlan.zhihu.com/p/64885176)

## Adadelta

## Adagrad

## Adam

## AdamW

## SparseAdam

## Adamax

## ASGD

## LBFGS

## RMSprop

## Rprop
该优化方法适用于full-batch，不适用于mini-batch，因而在min-batch大行其道的时代里，很少见到。

## SGD


# 调整学习率(torch.optim.lr_scheduler包)
`torch.optim.lr_scheduler`提供了几种epochs级别的学习率调整方法。

[torch.optim.lr_scheduler.ReduceLROnPlateau](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau)允许<font color=red>基于某些验证指标来动态的减少学习率</font>

使用方式:
```python
>>> scheduler = ...
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()
```

## LambdaLR
使用示例

```python
>>> # Assuming optimizer has two groups.
>>> lambda1 = lambda epoch: epoch // 30
>>> lambda2 = lambda epoch: 0.95 ** epoch
>>> scheduler = LambdaLR(optimizer, lr_lambda=[lambda1, lambda2])
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()
```

## MultiplicativeLR

将每个参数组的学习率乘以指定函数中给定的因子。 当last_epoch = -1时，将初始lr设置为lr。

```python
>>> lmbda = lambda epoch: 0.95
>>> scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()
```

## StepLR

在每个step_size时期，通过γ降低每个参数组的学习率。 注意，这种衰减可能与此调度程序外部的学习速率的其他更改同时发生。 当last_epoch = -1时，将初始lr设置为lr。

```python
>>> # Assuming optimizer uses lr = 0.05 for all groups
>>> # lr = 0.05     if epoch < 30
>>> # lr = 0.005    if 30 <= epoch < 60
>>> # lr = 0.0005   if 60 <= epoch < 90
>>> # ...
>>> scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()
```

## MultiStepLR
一旦epoch达到设定的milestones，则gamma进行衰减。
```python
>>> # Assuming optimizer uses lr = 0.05 for all groups
>>> # lr = 0.05     if epoch < 30
>>> # lr = 0.005    if 30 <= epoch < 80
>>> # lr = 0.0005   if epoch >= 80
>>> scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()


```

## ExponentialLR
```python
optimizer (Optimizer) – Wrapped optimizer.
gamma (float) – Multiplicative factor of learning rate decay.
last_epoch (int) – The index of last epoch. Default: -1.
verbose (bool) – If True, prints a message to stdout for each update. Default: False.
```

在每个epoch，衰减每个参数组的学习率。


## CosineAnnealingLR

复杂的consine函数。参见 [torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1, verbose=False)](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.CosineAnnealingLR)

## ReduceLROnPlateau
当指标停止改善时，降低学习率。 一旦学习停滞，模型通常会受益于将学习率降低2-10倍。 该调度程序读取一个指标数量，如果在“patience(耐心)”时期没有看到改善，则学习速度会降低。

```python
>>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
>>> scheduler = ReduceLROnPlateau(optimizer, 'min')
>>> for epoch in range(10):
>>>     train(...)
>>>     val_loss = validate(...)
>>>     # Note that step should be called after validate()
>>>     scheduler.step(val_loss)
```

## CyclicLR
带边界的学习率调整策略,比较复杂，@TODO：[torch.optim.lr_scheduler.CyclicLR](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.CyclicLR)

```python
>>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
>>> scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
>>> data_loader = torch.utils.data.DataLoader(...)
>>> for epoch in range(10):
>>>     for batch in data_loader:
>>>         train_batch(...)
>>>         scheduler.step()
```

## OneCycleLR

参考《Super-Convergence: Very Fast Training of Neural Networks Using Large Learning Rates》

@TODO


## CosineAnnealingWarmRestarts

consine算法。

## Stochastic Weight Averaging算法
@TODO
